In [1]:
import turicreate as tc
tc.config.set_num_gpus(-1)
tc.visualization.set_target(target='auto')

In [2]:
data_test = tc.SFrame('image_test_data/')
data_train = tc.SFrame('image_train_data/')

In [3]:
data_train.head()

id,image,label,deep_features,image_array
24,Height: 32 Width: 32,bird,"[0.242871761322,1.09545373917, 0.0, ...","[73.0, 77.0, 58.0, 71.0,68.0, 50.0, 77.0, 69.0, ..."
33,Height: 32 Width: 32,cat,"[0.525087952614, 0.0,0.0, 0.0, 0.0, 0.0, ...","[7.0, 5.0, 8.0, 7.0, 5.0,8.0, 5.0, 4.0, 6.0, 7.0, ..."
36,Height: 32 Width: 32,cat,"[0.566015958786, 0.0,0.0, 0.0, 0.0, 0.0, ...","[169.0, 122.0, 65.0,131.0, 108.0, 75.0, ..."
70,Height: 32 Width: 32,dog,"[1.12979578972, 0.0, 0.0,0.778194487095, 0.0, ...","[154.0, 179.0, 152.0,159.0, 183.0, 157.0, ..."
90,Height: 32 Width: 32,bird,"[1.71786928177, 0.0, 0.0,0.0, 0.0, 0.0, ...","[216.0, 195.0, 180.0,201.0, 178.0, 160.0, ..."
97,Height: 32 Width: 32,automobile,"[1.57818555832, 0.0, 0.0,0.0, 0.0, 0.0, ...","[33.0, 44.0, 27.0, 29.0,44.0, 31.0, 32.0, 45.0, ..."
107,Height: 32 Width: 32,dog,"[0.0, 0.0,0.220677852631, 0.0, ...","[97.0, 51.0, 31.0, 104.0,58.0, 38.0, 107.0, 61.0, ..."
121,Height: 32 Width: 32,bird,"[0.0, 0.23753464222, 0.0,0.0, 0.0, 0.0, ...","[93.0, 96.0, 88.0, 102.0,106.0, 97.0, 117.0, ..."
136,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 7.5737862587, 0.0, ...","[35.0, 59.0, 53.0, 36.0,56.0, 56.0, 42.0, 62.0, ..."
138,Height: 32 Width: 32,bird,"[0.658935725689, 0.0,0.0, 0.0, 0.0, 0.0, ...","[205.0, 193.0, 195.0,200.0, 187.0, 193.0, ..."


1. Computing summary statistics of the data: Sketch summaries are techniques for computing summary statistics of data very quickly. In GraphLab Create, SFrames and SArrays include a method:

1
.sketch_summary()
which computes such summary statistics. Using the training data, compute the sketch summary of the ‘label’ column and interpret the results. What’s the least common category in the training data? Save this result to answer the quiz at the end.


In [4]:
sketch = tc.Sketch(data_train['label'])
sketch


+------------------+-------+----------+
|       item       | value | is exact |
+------------------+-------+----------+
|      Length      |  2005 |   Yes    |
| # Missing Values |   0   |   Yes    |
| # unique values  |   4   |    No    |
+------------------+-------+----------+

Most frequent items:
+------------+-------+
|   value    | count |
+------------+-------+
| automobile |  509  |
|    cat     |  509  |
|    dog     |  509  |
|    bird    |  478  |
+------------+-------+


2. Creating category-specific image retrieval models: In most retrieval tasks, the data we have is unlabeled, thus we call these unsupervised learning problems. However, we have labels in this image dataset, and will use these to create one model for each of the 4 image categories, {‘dog’,’cat’,’automobile’,bird’}. To start, follow these steps:

Split the SFrame with the training data into 4 different SFrames. Each of these will contain data for 1 of the 4 categories above. Hint: if you use a logical filter to select the rows where the ‘label’ column equals ‘dog’, you can create an SFrame with only the data for images labeled ‘dog’.
Similarly to the image retrieval notebook you downloaded, you are going to create a nearest neighbor model using the 'deep_features' as the features, but this time create one such model for each category, using the corresponding subset of the training_data. You can call the model with the ‘dog’ data the dog_model, the one with the ‘cat’ data the cat_model, as so on.
You now have a nearest neighbors model that can find the nearest ‘dog’ to any image you give it, the dog_model; one that can find the nearest ‘cat’, the cat_model; and so on.

Using these models, answer the following questions. The cat image below is the first in the test data:


You can access this image, similarly to what we did in the iPython notebooks above, with this command:


1
image_test[0:1]
What is the nearest ‘cat’ labeled image in the training data to the cat image above (the first image in the test data)? Save this result.
Hint: When you query your nearest neighbors model, it will return a SFrame that looks something like this:

query_label	reference_label	distance	rank
0	34	42.9886641167	1
0	45	43.8444904098	2
0	251	44.2634660468	3
0	141	44.377719559	4
To understand each column in this table, see this documentation. For this question, the ‘reference_label’ column will be important, since it provides the index of the nearest neighbors in the dataset used to train it. (In this case, the subset of the training data labeled ‘cat’.)

What is the nearest ‘dog’ labeled image in the training data to the cat image above (the first image in the test data)? Save this result.

In [5]:
labels = ['dog','cat','automobile','bird']

In [6]:
frames_train = [data_train.filter_by(label, 'label', exclude=False) for label in labels]
frames_train[0]

id,image,label,deep_features,image_array
70,Height: 32 Width: 32,dog,"[1.12979578972, 0.0, 0.0,0.778194487095, 0.0, ...","[154.0, 179.0, 152.0,159.0, 183.0, 157.0, ..."
107,Height: 32 Width: 32,dog,"[0.0, 0.0,0.220677852631, 0.0, ...","[97.0, 51.0, 31.0, 104.0,58.0, 38.0, 107.0, 61.0, ..."
177,Height: 32 Width: 32,dog,"[0.0, 1.45965671539, 0.0,0.422992348671, 0.0, ...","[55.0, 75.0, 42.0, 51.0,76.0, 37.0, 57.0, 83.0, ..."
424,Height: 32 Width: 32,dog,"[0.942399680614, 0.0,0.220352768898, 0.0, ...","[60.0, 35.0, 18.0, 63.0,49.0, 38.0, 66.0, 56.0, ..."
462,Height: 32 Width: 32,dog,"[1.43462562561, 0.0, 0.0,0.0, 0.0, 0.0, ...","[86.0, 69.0, 75.0, 57.0,41.0, 48.0, 46.0, 35.0, ..."
542,Height: 32 Width: 32,dog,"[0.451547086239, 0.0,0.0, 0.0, 0.0, 0.0, ...","[196.0, 174.0, 113.0,140.0, 117.0, 65.0, 8 ..."
573,Height: 32 Width: 32,dog,"[0.592360973358, 0.0,0.0, 0.0, 0.0, 0.0, ...","[153.0, 103.0, 52.0,151.0, 102.0, 49.0, ..."
851,Height: 32 Width: 32,dog,"[0.690123438835, 0.0,0.0, 0.0, 0.305860161 ...","[39.0, 6.0, 4.0, 53.0,23.0, 24.0, 57.0, 37.0, ..."
919,Height: 32 Width: 32,dog,"[0.0, 0.177558660507,0.139396846294, 0.0, ...","[29.0, 43.0, 4.0, 24.0,35.0, 6.0, 24.0, 37.0, ..."
1172,Height: 32 Width: 32,dog,"[0.517601490021, 0.0,1.96418333054, 0.0, 0.0, ...","[182.0, 180.0, 197.0,196.0, 192.0, 209.0, ..."


In [7]:
knn_model = [tc.nearest_neighbors.create(frame,features=['deep_features'],label='id') for frame in frames_train]

Starting brute force nearest neighbors model training.

Starting brute force nearest neighbors model training.

Starting brute force nearest neighbors model training.

Starting brute force nearest neighbors model training.

In [8]:
cat = data_test[0:1]
cat['image'].explore()

Materializing SFrame

In [9]:
# image_test[0:1]
knn_model[1].query(cat,k=1)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 5.611ms      |

| Done         |         | 100         | 20.006ms     |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,16289,34.623719208,1


In [10]:
def get_images_from_ids(query_result):
    return data_train.filter_by(query_result['reference_label'],'id')

In [11]:
cat_neighbors = get_images_from_ids(knn_model[1].query(cat,k=1))

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 3.556ms      |

| Done         |         | 100         | 19.872ms     |

+--------------+---------+-------------+--------------+

In [12]:
cat_neighbors['image'].explore()

Materializing SFrame

In [13]:
knn_model[0].query(cat,k=1)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 5.869ms      |

| Done         |         | 100         | 23.623ms     |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,16976,37.4642628784,1


In [14]:
dog_neighbours = get_images_from_ids(knn_model[0].query(cat,k=1))

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 3.523ms      |

| Done         |         | 100         | 24.605ms     |

+--------------+---------+-------------+--------------+

In [15]:
dog_neighbours['image'].explore()

Materializing SFrame

3. A simple example of nearest-neighbors classification: When we queried a nearest neighbors model, the ‘distance’ column in the table above shows the computed distance between the input and each of the retrieved neighbors. In this question, you will use these distances to perform a classification task, using the idea of a nearest-neighbors classifier.

For the first image in the test data (image_test[0:1]), which we used above, compute the mean distance between this image at its 5 nearest neighbors that were labeled ‘cat’ in the training data (similarly to what you did in the previous question). Save this result.
Similarly, for the first image in the test data (image_test[0:1]), which we used above, compute the mean distance between this image at its 5 nearest neighbors that were labeled ‘dog’ in the training data (similarly to what you did in the previous question). Save this result.
On average, is the first image in the test data closer to its 5 nearest neighbors in the ‘cat’ data or in the ‘dog’ data? (In a later course, we will see that this is an example of what is called a k-nearest neighbors classifier, where we use the label of neighboring points to predict the label of a test point.)

In [16]:
knn_model[1].query(cat,k=1)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 8.726ms      |

| Done         |         | 100         | 25.451ms     |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,16289,34.623719208,1


In [17]:
mean_distance = knn_model[1].query(cat,k=5)['distance'].mean()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 4.276ms      |

| Done         |         | 100         | 21.716ms     |

In [18]:
print 'Cat mean distance: ' + str(mean_distance)

+--------------+---------+-------------+--------------+

Cat mean distance: 36.1557307098


In [19]:
mean_distance = knn_model[0].query(cat,k=5)['distance'].mean()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 5.821ms      |

| Done         |         | 100         | 30.426ms     |

+--------------+---------+-------------+--------------+

In [20]:
print 'Dog mean distance: ' + str(mean_distance)

Dog mean distance: 37.7707113618


In [21]:
# closer to cat KNN

4. [Challenging Question] Computing nearest neighbors accuracy using SFrame operations: A nearest neighbor classifier predicts the label of a point as the most common label of its nearest neighbors. In this question, we will measure the accuracy of a 1-nearest-neighbor classifier, i.e., predict the output as the label of the nearest neighbor in the training data. Although there are simpler ways of computing this result, we will go step-by-step here to introduce you to more concepts in nearest neighbors and SFrames, which will be useful later in this Specialization.

Training models: For this question, you will need the nearest neighbors models you learned above on the training data, i.e., the dog_model, cat_model, automobile_model and bird_model.
Spliting test data by label: Above, you split the train data SFrame into one SFrame for images labeled ‘dog’, another for those labeled ‘cat’, etc. Now, do the same for the test data. You can call the resulting SFrames

1
image_test_cat, image_test_dog, image_test_bird, image_test_automobile
Finding nearest neighbors in the training set for each part of the test set: Thus far, we have queried, e.g.,

1
dog_model.query()
our nearest neighbors models with a single image as the input, but you can actually query with a whole set of data, and it will find the nearest neighbors for each data point. Note that the input index will be stored in the ‘query_label’ column of the output SFrame.

Using this knowledge find the closest neighbor in to the dog test data using each of the trained models, e.g.,


1
dog_cat_neighbors = cat_model.query(image_test_dog, k=1)
finds 1 neighbor (that’s what k=1 does) to the dog test images (image_test_dog) in the cat portion of the training data (used to train the cat_model).

Now, do this for every combination of the labels in the training and test data.

Create an SFrame with the distances from ‘dog’ test examples to the respective nearest neighbors in each class in the training data: The ‘distance’ column in dog_cat_neighbors above contains the distance between each ‘dog’ image in the test set and its nearest ‘cat’ image in the training set. The question we want to answer is how many of the test set ‘dog’ images are closer to a ‘dog’ in the training set than to a ‘cat’, ‘automobile’ or ‘bird’. So, next we will create an SFrame containing just these distances per data point. The goal is to create an SFrame called dog_distances with 4 columns:
i. dog_distances[‘dog-dog’] ---- storing dog_dog_neighbors[‘distance’]

ii. dog_distances[‘dog-cat’] ---- storing dog_cat_neighbors[‘distance’]

iii. dog_distances[‘dog-automobile’] ---- storing dog_automobile_neighbors[‘distance’]

iv. dog_distances[‘dog-bird’] ---- storing dog_bird_neighbors[‘distance’]

Hint: You can create a new SFrame from the columns of other SFrames by creating a dictionary with the new columns, as shown in this example:


1
new_sframe = graphlab.SFrame({'foo': other_sframe['foo'],'bar': 
  some_other_sframe['bar']})
The resulting SFrame will look something like this:

dog-automobile	dog-bird	dog-cat	dog-dog
41.9579761457	41.7538647304	36.4196077068	33.4773590373
46.0021331807	41.3382958925	38.8353268874	32.8458495684
42.9462290692	38.6157590853	36.9763410854	35.0397073189
Computing the number of correct predictions using 1-nearest neighbors for the dog class: Now that you have created the SFrame dog_distances, you will learn to use the method

1
.apply()
on this SFrame to iterate line by line and compute the number of ‘dog’ test examples where the distance to the nearest ‘dog’ was lower than that to the other classes. You will do this in three steps:

i. Consider one row of the SFrame dog_distances. Let’s call this variable row. You can access each distance by calling, for example,


1
row['dog-cat']
which, in example table above, will have value equal to 36.4196077068 for the first row.

Create a function starting with


1
def is_dog_correct(row):
which returns 1 if the value for row[‘dog-dog’] is lower than that of the other columns, and 0 otherwise. That is, returns 1 if this row is correctly classified by 1-nearest neighbors, and 0 otherwise.

ii. Using the function is_dog_correct(row), you can check if 1 row is correctly classified. Now, you want to count how many rows are correctly classified. You could do a for loop iterating through each row and applying the function is_dog_correct(row). This method will be really slow, because the SFrame is not optimized for this type of operation.

Instead, we will use the .apply() method to iterate the function is_dog_correct for each row of the SFrame. Read about using the .apply() method here.

iii. Computing the number of correct predictions for ‘dog’: You can now call:


1
dog_distances.apply(is_dog_correct)
which will return an SArray (a column of data) with a 1 for every correct row and a 0 for every incorrect one. You can call:


1
.sum()
on the result to get the total number of correctly classified ‘dog’ images in the test set!

Hint: To make sure your code is working correctly, if you were to do the two steps above in this question to count the number of correctly classified ‘cat’ images in the test data, instead of ‘dog’, the result would be 548.

Accuracy of predicting dog in the test data: Using the work you did in this question, what is the accuracy of the 1-nearest neighbor classifier at classifying ‘dog’ images from the test set? Save this result to answer the quiz at the end.

In [22]:
frames_test = [data_test.filter_by(label, 'label', exclude=False) for label in labels]
frames_test[0]

id,image,label,deep_features,image_array
12,Height: 32 Width: 32,dog,"[0.322317481041, 0.0,1.24933350086, 0.0, 0.0, ...","[91.0, 64.0, 30.0, 82.0,58.0, 30.0, 87.0, 73.0, ..."
16,Height: 32 Width: 32,dog,"[0.0, 0.0,0.347357034683, 0.0, ...","[95.0, 76.0, 78.0, 92.0,77.0, 78.0, 89.0, 77.0, ..."
24,Height: 32 Width: 32,dog,"[1.31557655334, 0.0, 0.0,0.0, 0.0, 0.0, ...","[136.0, 134.0, 118.0,142.0, 141.0, 126.0, ..."
31,Height: 32 Width: 32,dog,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 9.26018810272, 0.0, ...","[127.0, 130.0, 81.0,130.0, 133.0, 88.0, ..."
33,Height: 32 Width: 32,dog,"[0.130786716938,0.727667212486, 0.0, ...","[118.0, 113.0, 81.0,122.0, 117.0, 83.0, ..."
39,Height: 32 Width: 32,dog,"[0.0112721920013, 0.0,0.76088231802, 0.0, 0.0, ...","[72.0, 88.0, 97.0, 55.0,70.0, 84.0, 61.0, 73.0, ..."
42,Height: 32 Width: 32,dog,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 8.33566474915, 0.0, ...","[149.0, 149.0, 150.0,149.0, 149.0, 146.0, ..."
85,Height: 32 Width: 32,dog,"[0.0, 0.298464655876,0.0, 0.0, 0.0, 0.0, ...","[113.0, 174.0, 228.0,113.0, 174.0, 227.0, ..."
101,Height: 32 Width: 32,dog,"[0.591187477112, 0.0,0.0, 0.0, 0.420105159 ...","[60.0, 77.0, 93.0, 60.0,76.0, 92.0, 58.0, 74.0, ..."
128,Height: 32 Width: 32,dog,"[0.0, 0.0,0.0717121362686, ...","[86.0, 18.0, 39.0, 84.0,18.0, 38.0, 82.0, 16.0, ..."


In [23]:
# dog
knn_model[0].query(frames_test[0])

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 8

number of query data blocks: 1

+--------------+---------+-------------+--------------+

...

+--------------+---------+-------------+--------------+

| 1000         | 64000   | 12.5737     | 135.443ms    |

| Done         | 509000  | 100         | 151.562ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,49803,33.4773590373,1
0,21235,34.415221599,2
0,23803,34.8138630061,3
0,41752,34.9289313468,4
0,13865,37.1546409194,5
1,5755,32.8458495684,1
1,38013,35.6379572518,2
1,10669,37.0042463585,3
1,11933,37.0051632125,4
1,48566,37.9279031587,5


In [24]:
# cat
knn_model[1].query(frames_test[1])

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 8

number of query data blocks: 1

+--------------+---------+-------------+--------------+

...

+--------------+---------+-------------+--------------+

| 1000         | 63000   | 12.3772     | 103.66ms     |

| Done         | 509000  | 100         | 130.14ms     |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,16289,34.623719208,1
0,45646,36.0068799284,2
0,32139,36.5200813436,3
0,25713,36.7548502521,4
0,331,36.8731228168,5
1,13094,33.8680579302,1
1,10883,34.4414068951,2
1,43295,34.8348775045,3
1,6304,34.9159478835,4
1,8302,35.1367177322,5


In [25]:
# automobile
knn_model[2].query(frames_test[2])

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 8

number of query data blocks: 1

+--------------+---------+-------------+--------------+

...

+--------------+---------+-------------+--------------+

| 1000         | 63000   | 12.3772     | 114.419ms    |

| Done         | 509000  | 100         | 130.35ms     |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,45011,36.8738683312,1
0,6851,38.1935095581,2
0,35863,39.9191107179,3
0,18456,41.042930668,4
0,42292,41.8291652654,5
1,16962,34.6192710221,1
1,34588,34.9094073056,2
1,9056,36.1488313292,3
1,23332,37.5389434824,4
1,22788,38.2783480446,5


In [26]:
# bird
knn_model[3].query(frames_test[3])

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 8

number of query data blocks: 1

+--------------+---------+-------------+--------------+

...

+--------------+---------+-------------+--------------+

| 1000         | 60000   | 12.5523     | 98.804ms     |

| Done         | 478000  | 100         | 131.214ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,46242,51.3719536389,1
0,24734,51.4769680265,2
0,44732,52.599061288,3
0,8438,53.0556794136,4
0,5892,53.4472261033,5
1,649,41.6659333448,1
1,12191,43.308744363,2
1,44634,43.6934459805,3
1,14946,44.3550773945,4
1,49573,44.4700173468,5


In [27]:
dog_dog_neighbors = knn_model[0].query(frames_test[0], k=1)
dog_cat_neighbors = knn_model[1].query(frames_test[0], k=1)
dog_automobile_neighbors = knn_model[2].query(frames_test[0], k=1)
dog_bird_neighbors = knn_model[3].query(frames_test[0], k=1)

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 8

number of query data blocks: 1

+--------------+---------+-------------+--------------+

...

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 1000         | 60000   | 12.5523     | 102.507ms    |

| Done         | 478000  | 100         | 133.277ms    |

+--------------+---------+-------------+--------------+

In [28]:
dog_cat_neighbors

query_label,reference_label,distance,rank
0,33,36.4196077068,1
1,30606,38.8353268874,1
2,5545,36.9763410854,1
3,19631,34.5750072914,1
4,7493,34.778824791,1
5,47044,35.1171578292,1
6,13918,40.6095830913,1
7,10981,39.9036867306,1
8,45456,38.0674700168,1
9,44673,42.7258732951,1


In [29]:
new_sframe = tc.SFrame({
    'dog-automobile': dog_automobile_neighbors['distance'],
    'dog-bird': dog_bird_neighbors['distance'],
    'dog-cat': dog_cat_neighbors['distance'],
    'dog-dog': dog_dog_neighbors['distance'],
})
new_sframe

dog-automobile,dog-bird,dog-cat,dog-dog
41.9579761457,41.7538647304,36.4196077068,33.4773590373
46.0021331807,41.3382958925,38.8353268874,32.8458495684
42.9462290692,38.6157590853,36.9763410854,35.0397073189
41.6866060048,37.0892269954,34.5750072914,33.9010327697
39.2269664935,38.272288694,34.778824791,37.4849250909
40.5845117698,39.1462089236,35.1171578292,34.945165344
45.1067352961,40.523040106,40.6095830913,39.0957278345
41.3221140974,38.1947918393,39.9036867306,37.7696131032
41.8244654995,40.1567131661,38.0674700168,35.1089144603
45.4976929401,45.5597962603,42.7258732951,43.2422832585


In [30]:
predictions = new_sframe.apply(lambda x: True if x['dog-dog'] < x['dog-cat'] and x['dog-dog'] < x['dog-automobile'] and x['dog-dog'] < x['dog-bird'] else False)

In [31]:
predictions.sum()

678

In [32]:
cat_dog_neighbors = knn_model[0].query(frames_test[1], k=1)
cat_cat_neighbors = knn_model[1].query(frames_test[1], k=1)
cat_automobile_neighbors = knn_model[2].query(frames_test[1], k=1)
cat_bird_neighbors = knn_model[3].query(frames_test[1], k=1)
new_sframe = tc.SFrame({
    'cat-automobile': cat_automobile_neighbors['distance'],
    'cat-bird': cat_bird_neighbors['distance'],
    'cat-cat': cat_cat_neighbors['distance'],
    'cat-dog': cat_dog_neighbors['distance'],
})
predictions = new_sframe.apply(lambda x: True if x['cat-cat'] < x['cat-dog'] and x['cat-cat'] < x['cat-automobile'] and x['cat-cat'] < x['cat-bird'] else False)
predictions.sum()

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 8

number of query data blocks: 1

+--------------+---------+-------------+--------------+

...

+--------------+---------+-------------+--------------+

| 1000         | 60000   | 12.5523     | 104.576ms    |

| Done         | 478000  | 100         | 128.167ms    |

+--------------+---------+-------------+--------------+

548

In [33]:
# 548 matches HINT given for cat comparison.

In [34]:
len(frames_test[0])

1000

In [35]:
from __future__ import division
678/1000

0.678